In [1]:
import numpy as np
import gurobipy as gp
from scipy.optimize import minimize
import pandas as pd

In [2]:
train = pd.read_csv('training_data.csv')
test = pd.read_csv('test_data.csv')

In [3]:
train.head()

,y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50
0,8.536145,-1.535413,0.718888,-2.099149,-0.442842,-0.598978,-1.642574,0.207755,0.760642,0.575874,...,0.361866,1.793098,-0.631287,-0.061751,0.511049,0.488754,-0.612772,-0.471045,-1.139781,-0.260773
1,4.808344,-1.734609,0.551981,-2.147673,-1.552944,1.514910,-1.143972,0.737594,1.321243,-0.261684,...,-0.677985,-0.165679,0.065405,0.137162,1.258197,-0.120834,-1.564834,-0.242565,-0.001827,1.187453
2,-1.530427,0.097257,0.107634,-0.194222,0.335454,-0.408199,0.133265,0.706179,0.394971,-0.437116,...,1.108801,0.333791,0.282055,-1.086294,-0.115354,0.257857,-0.088838,-0.751231,1.450609,0.290593
3,-0.428243,-0.067702,0.557836,0.700848,-1.121376,1.722274,0.613525,0.700909,-0.417976,1.069749,...,0.692511,-0.350990,0.624558,0.434520,-0.367409,-1.144681,-0.136524,-0.557214,0.416303,0.484495
4,0.566694,0.488729,0.211483,0.568389,0.646837,0.163868,-0.002152,0.125137,0.493571,1.705451,...,-0.000605,1.075280,0.182281,-1.138458,0.106092,0.544640,-0.383487,-0.425773,2.667647,-0.050748


In [ ]:
#decision variables: betas 0-50 (C) 
#decision variables: z 1-50 (B)
#obj: min 

In [45]:
def MIQP_variable(X,y,M,k):
    
    m = X.shape[1]-1
    n=X.shape[0]
    
    x0 = np.ones((n,1))
    X = np.asmatrix(np.hstack((x0,X[:,1:])))
    
    Q = np.zeros((2*m+1,2*m+1))
    Q[0:m+1,0:m+1]=X.T@X
    Y = np.zeros(2*m+1)
    Y[0:m+1]=-2*y@X

    #constraints
    A = np.zeros((2*m+1,2*m+1))
    b = np.zeros(2*m+1)
    sense = np.array(['']*(2*m+1))
    
    #sum z ≤ k 
    A[0,m+1:]=1
    b[0]=k
    sense[0]="<"
    
    #M
    for i in range(1,m+1):
        A[i,[i,m+i]]=[1,-M]
        b[i]=0
        sense[i]="<"
    
        A[i+m,[i,m+i]]=[1,M]
        b[i+m]=0
        sense[i+m]=">"
   
    QPMod = gp.Model()
    QPMod_x = QPMod.addMVar(2*m+1,vtype=['C']*(m+1)+['B']*m,lb=np.array([-np.inf]*(m + 1) + [0]*m))
    QPMod_con = QPMod.addMConstrs(A, QPMod_x, sense, b)
    QPMod.setMObjective(Q,Y,0,sense=gp.GRB.MINIMIZE)
    QPMod.Params.OutputFlag = 0 
    QPMod.Params.TimeLimit = 60

    QPMod.optimize()
    
    return QPMod

In [21]:
# CV

In [44]:
def cross_validation(data,K_num):
    cv_list = []
    
    kf = KFold(n_splits=K_num)
    # getting train and validation index
    for train_idx, val_idx in kf.split(data):
        cv_list.append((train_idx, val_idx))
    
    return cv_list

In [32]:
def get_sum_squared_error(y_true,y_pred):
    return (y_true-y_pred).T @ (y_true-y_pred)

In [24]:
#data setup
X_train = np.zeros((train.shape[0], train.shape[1]))
X_train[:, 0] = 1
X_train[:, 1:] = train.iloc[:, 1:].values
y_train = train['y'].values

In [46]:
#MIQP
sum_error = []
K_list = np.arange(5,51,5) # list of k
cv_index_list = cross_validation(X_train,10) #create 10 CV

results_MIQP = pd.DataFrame(data=None, columns=['k', 'cross_val_error'])
beta_lists = []

for K in K_list:
    error = []
    beta_list = []
    
    for train_index, val_index in cv_index_list:
        x, y = X_train[train_index], y_train[train_index]
        x_val, y_val = X_train[val_index], y_train[val_index]
        
        MIQPModel = MIQP_variable(x,y,50,K)
        beta_list.append(MIQPModel.x[:x.shape[1]])
        
        predicted_y = np.array(MIQPModel.x[:x.shape[1]])@x_val.T
        error.append(get_sum_squared_error(y_val,predicted_y))
    
    sum_error.append(np.sum(error))
    beta_lists.append(beta_list)

results_MIQP['k'] = K_list
results_MIQP['cross_val_error'] = sum_error
results_MIQP.to_csv('results_MIQP.csv')


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Deprecated, use Model.addMConstr() instead


In [ ]:
#((y-model.x[:m+1]@X.T).T @ (y-model.x[:m+1]@X.T))[1]

In [47]:
results_MIQP

,k,cross_val_error
0,5,917.479061
1,10,724.787631
2,15,763.721994
3,20,796.621825
4,25,781.250142
5,30,830.082402
6,35,831.104008
7,40,846.940858
8,45,846.958615
9,50,847.184545


In [49]:
#with test data

In [50]:
X_test = np.zeros((test.shape[0], test.shape[1]))
X_test[:, 0] = 1
X_test[:, 1:] = test.iloc[:, 1:].values
y_test = test['y'].values

In [65]:
MIQPModel = MIQP_variable(X_train,y_train,50,10)
test_y_pred = np.array(MIQPModel.x[:x.shape[1]])@X_test.T
test_error = get_sum_squared_error(y_test,test_y_pred)
test_results_MIQP = pd.DataFrame({'y_true': y_test, 'y_predicted': test_y_pred})


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  """Entry point for launching an IPython kernel.


In [72]:
MIQPModel.x

TypeError: list indices must be integers or slices, not list

In [73]:
MIQP_test_betas = []
for idx in res[0:11]:
    MIQP_test_betas.append(MIQPModel.x[idx])

In [67]:
res = [idx for idx, val in enumerate(MIQPModel.x) if val != 0]

In [70]:
res[0:11]

[0, 9, 15, 16, 23, 24, 26, 34, 45, 47, 48]

In [74]:
MIQP_test_betas

[0.9725240765514854,
 -2.308207261938351,
 -0.5183261233652071,
 -0.2041620134665432,
 -1.5591431785888525,
 0.8669733628924494,
 -1.3119194151226843,
 0.408165303305882,
 1.781474891076474,
 0.8873829240125524,
 -0.28229212764205913]

In [83]:
betas_MIQP_test = pd.DataFrame(data=None, columns=['index', 'coefficient'])
betas_MIQP_test['index']=res[0:11]
betas_MIQP_test['coefficient']=MIQP_test_betas
betas_MIQP_test = betas_MIQP_test.T
betas_MIQP_test.columns = betas_MIQP_test.iloc[0]
betas_MIQP_test = betas_MIQP_test[1:]
betas_MIQP_test


index,0.0,9.0,15.0,16.0,23.0,24.0,26.0,34.0,45.0,47.0,48.0
coefficient,0.972524,-2.308207,-0.518326,-0.204162,-1.559143,0.866973,-1.311919,0.408165,1.781475,0.887383,-0.282292


In [38]:
# LASSO
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold

In [42]:
sum_error2 = []
lambda_list = [0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000]
results_lasso = pd.DataFrame(data=None, columns=['lambda', 'cross_val_error'])
beta_lists2 = []
cv_index_list = cross_validation(X_train,10)

for Lambda in lambda_list:
    error = []
    beta_list = []
    
    for train_index, val_index in cross_val_list:
        x, y = X_train[train_index], y_train[train_index]
        x_val, y_val = X_train[val_index], y_train[val_index]
        
        lasso = Lasso(alpha=Lambda)
        lasso.fit(x, y)
        
        predicted_y = lasso.predict(x_val)
        error.append(get_sum_squared_error(y_val,predicted_y))
    
    sum_error2.append(np.sum(error))
    beta_lists2.append(beta_list)

results_lasso['lambda'] = lambda_list
results_lasso['cross_val_error'] = sum_error2
results_lasso.to_csv('results_lasso.csv')


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.128e+02, tolerance: 4.374e-01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.260e+01, tolerance: 4.528e-01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasin

In [43]:
results_lasso

,lambda,cross_val_error
0,0.00001,847.002294
1,0.00010,845.983748
2,0.00100,836.904951
3,0.01000,778.400013
4,0.10000,699.644522
5,1.00000,2106.081819
6,10.00000,4943.255428
7,100.00000,4943.255428
8,1000.00000,4943.255428


In [75]:
lasso = Lasso(alpha=0.0764)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

In [76]:
test_error

116.82719822762621

In [84]:
test_error = get_sum_squared_error(y_test,y_pred_lasso)
test_results_lasso = pd.DataFrame({'y_true': y_test, 'y_predicted': y_pred_lasso})


In [78]:
lasso.coef_

array([ 0.        , -0.        , -0.        ,  0.        ,  0.        ,
       -0.        ,  0.        , -0.        , -0.        , -2.16053214,
        0.        , -0.0596295 , -0.        , -0.        , -0.        ,
       -0.4191218 , -0.19325213,  0.        ,  0.        , -0.        ,
        0.        ,  0.        , -0.19518029, -1.36387765,  0.74258542,
       -0.        , -1.30481221, -0.        ,  0.        ,  0.05798716,
        0.        , -0.        ,  0.        , -0.09737622,  0.2833979 ,
        0.        ,  0.        ,  0.        ,  0.        , -0.23156121,
        0.        , -0.        ,  0.        ,  0.        ,  0.03076854,
        1.56360851, -0.02158588,  0.6999003 , -0.0928903 ,  0.        ,
        0.        ])

In [85]:
res = [idx for idx, val in enumerate(lasso.coef_) if val != 0]

In [87]:
Lasso_test_betas = []
for idx in res:
    Lasso_test_betas.append(lasso.coef_[idx])

In [89]:
betas_Lasso_test = pd.DataFrame(data=None, columns=['index', 'coefficient'])
betas_Lasso_test['index']=res
betas_Lasso_test['coefficient']=Lasso_test_betas
betas_Lasso_test = betas_Lasso_test.T
betas_Lasso_test.columns = betas_Lasso_test.iloc[0]
betas_Lasso_test = betas_Lasso_test[1:]
betas_Lasso_test.to_csv('betas_Lasso_test.csv')